In [1]:
%load_ext autoreload
%autoreload 2

%config InteractiveShell.ast_node_interactivity='all'
%config InteractiveShell.ast_node_interactivity='all'

import sys
sys.path.insert(1, '../')


# Identify unique accounts in list
The purpose of this script is to provide a listing of all accounts from multiiple account listings across perioods for a company, and to provide these in the correct order that they should appear.

Ideally this would not be required and all accounts would be provided as a single listing from the accounting software used. However, this list isn't always made available and accounts often change from period to period. Taking all unique accounts from all the lists combined also doesnt maintain the order required. This script with therefore obtain all unique accounts and maintain them in an acceptable order of accounts.

Rrequirements:
* You will need to provide an Excel Sheet in the `data` folder where each sheet represents a separate account listing - each sheet must have the same number of columns ordered from highest grouping to lowest. [`TO DO`: Make it so multiple account levels can be provided].
* The sheet name should refer to the applicable period for that list.
* There must be no duplicate account groupings in any one list. e.g. [Asset, Current Asset, Financial (call option)] and [Liability, Current Liability, Financial (call option)] are fine event though the accounts are the same, but having [Asset, Current Asset, Financial (call option)] listed twice in the same listing would be an issue.
* The accounts are in the correct order, with no instances where accounts are displayed in a different order. For example if one list is A B C, you cant have another list A C B as C and D are in different orders in either listing.

Potential issues
There can be instances where the order determined is not technically correct, for example, if  the following lists are provided:
* 1:	A	B	D	G		
* 2:	A	B	C	E	F	
* 3:	A	B	C	E	F	G
Since CEF and D are never in the same list, you only know that D is before G and after B, and CEF are before G and after B. Therefore a valid result is: ['A', 'B', 'C', 'E', 'D', 'F', 'G'].

If the lists were:
* 1:	A	B	D	G		
* 2:	X	Y	Z
Since there is no overlapping values, a valid result is: ['X', 'Y', 'Z', 'A', 'B', 'D', 'G', ]

These issues are a limitation of the information available. Therefore it is best to provide as many lists as possible to ensure all cases are covered.

If you want to run a new list you current complete list, it is best to start from scratch again and provide all lists. This is due to the above mentioned issue and the potential for ordering logic to be incorrect.

# Setup
Provide the file to use.

Conditions for the file:
* one sheet per account listing
* the account should be listed in the first column
* there needs to be now empty rows between any accounts or the heading

In [6]:
from utils import absolute_path

file_name = absolute_path("get_unique_accounts\\account_lists.xlsx")
file_name

'C:\\Users\\tyewf\\github_projects\\excel_accounting_data_analysis\\get_unique_accounts\\account_lists.xlsx'

# Run Complete Script
Runs all checks and produces all outputs in one go.

In [8]:
from account_list_generator import main

main(file_name = file_name)

2024-02-04 10:23:10,946 - DEBUG - 


Running: Main
2024-02-04 10:23:10,946 - DEBUG - 


Running: assert_file_extension
2024-02-04 10:23:10,950 - DEBUG - 


Running: read_sheets_as_df
2024-02-04 10:23:10,953 - DEBUG - 


Running: basic_checks_acc_gen
2024-02-04 10:23:10,954 - DEBUG - All checks have passes.
2024-02-04 10:23:10,954 - DEBUG - 


Running: get_list_of_unique_accounts
2024-02-04 10:23:10,955 - DEBUG - account_lists_dict:{'Sheet1': ['A', 'B', 'D', 'G'], 'Sheet2': ['A', 'B', 'C', 'E', 'F'], 'Sheet3': ['A', 'B', 'C', 'E', 'F', 'G']}
2024-02-04 10:23:10,955 - DEBUG - list of all unique accounts:['B', 'G', 'F', 'D', 'A', 'E', 'C']
2024-02-04 10:23:10,956 - DEBUG - 


Running: generate_check_account_order_dict
2024-02-04 10:23:10,956 - DEBUG - Current account listing:Sheet1
2024-02-04 10:23:10,956 - DEBUG - Current account listing:Sheet2
2024-02-04 10:23:10,957 - DEBUG - Current account listing:Sheet3
2024-02-04 10:23:10,957 - DEBUG - check_account_order_dict:{'B': {'before': ['A'

['A', 'B', 'C', 'E', 'D', 'F', 'G']

# Run Script step by step
This should follow all required steps from the model, but allows you to execute it step by step.

In [24]:
import pandas as pd
import os
import logging
import logging.config
from datetime import datetime
from openpyxl import Workbook
from contextlib import contextmanager

# from utils import absolute_path
# from utils import setup_logging
# from utils import assert_file_extension
# 
# from account_list_generator import read_sheets_as_df
# from account_list_generator import basic_checks_acc_gen
# from account_list_generator import get_list_of_unique_accounts
# from account_list_generator import generate_check_account_order_dict
# from account_list_generator import complete_account_order_checker
# from account_list_generator import correctly_ordered_list
# from account_list_generator import document_account_lists

[autoreload of utils failed: Traceback (most recent call last):
  File "C:\Users\tyewf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\tyewf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\extensions\autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2032.0_x64__qbz5n2kfra8p0\Lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "c:\Users\tyewf\gith

In [7]:
# Create a logger variable
logger = logging.getLogger(__name__)
logger.info('Logging initiated.')

2024-02-04 09:25:10,895 - INFO - Running yaml logger
2024-02-04 09:25:10,897 - INFO - Logging initiated.


In [16]:
# Check file extension is correct
assert_file_extension(file_name, expected_extension='.xlsx')

NameError: name 'assert_file_extension' is not defined

In [9]:
# Read the Excel file
xls = pd.ExcelFile(file_name)

In [10]:
# Read in sheets as dfs
account_list_dfs_dict = read_sheets_as_df(xls)
account_list_dfs_dict

2024-02-04 09:25:14,348 - DEBUG - 


Running: read_sheets_as_df


{'Sheet1':   list_1
 0      A
 1      B
 2      D
 3      G,
 'Sheet2':   list_2
 0      A
 1      B
 2      C
 3      E
 4      F,
 'Sheet3':   list_3
 0      A
 1      B
 2      C
 3      E
 4      F
 5      G}

In [11]:
basic_checks_acc_gen(account_list_dfs_dict=account_list_dfs_dict)

2024-02-04 09:25:16,685 - DEBUG - 


Running: basic_checks_acc_gen
2024-02-04 09:25:16,685 - DEBUG - All checks have passes.


True

In [12]:
# Get a list of unique accounts from all lists
account_lists_dict, unique_accounts_list=get_list_of_unique_accounts(account_list_dfs_dict)

2024-02-04 09:25:17,715 - DEBUG - 


Running: get_list_of_unique_accounts
2024-02-04 09:25:17,716 - DEBUG - account_lists_dict:{'Sheet1': ['A', 'B', 'D', 'G'], 'Sheet2': ['A', 'B', 'C', 'E', 'F'], 'Sheet3': ['A', 'B', 'C', 'E', 'F', 'G']}
2024-02-04 09:25:17,717 - DEBUG - list of all unique accounts:['C', 'G', 'F', 'E', 'A', 'B', 'D']


In [13]:
check_account_order_dict=generate_check_account_order_dict(account_lists_dict, unique_accounts_list)

2024-02-04 09:25:18,941 - DEBUG - 


Running: generate_check_account_order_dict
2024-02-04 09:25:18,941 - DEBUG - Current account listing:Sheet1
2024-02-04 09:25:18,942 - DEBUG - Current account listing:Sheet2
2024-02-04 09:25:18,942 - DEBUG - Current account listing:Sheet3
2024-02-04 09:25:18,942 - DEBUG - check_account_order_dict:{'C': {'before': ['A', 'B'], 'after': ['G', 'E', 'F']}, 'G': {'before': ['C', 'F', 'E', 'A', 'B', 'D'], 'after': []}, 'F': {'before': ['C', 'A', 'B', 'E'], 'after': ['G']}, 'E': {'before': ['C', 'A', 'B'], 'after': ['G', 'F']}, 'A': {'before': [], 'after': ['C', 'G', 'F', 'E', 'B', 'D']}, 'B': {'before': ['A'], 'after': ['C', 'G', 'F', 'E', 'D']}, 'D': {'before': ['A', 'B'], 'after': ['G']}}


In [14]:
# Check all account lists are in the expected order
assert complete_account_order_checker(
        check_account_order_dict,
        account_lists_dict,
), 'Some accounts are not in the correctr order, please review'

# At this point you have a list of acount lists
# These account lists are all in the correct order

2024-02-04 09:25:20,156 - DEBUG - 


Running: complete_account_order_checker
2024-02-04 09:25:20,156 - DEBUG - Checking list: Sheet1
2024-02-04 09:25:20,157 - DEBUG - 


Running: account_order_checker
2024-02-04 09:25:20,157 - DEBUG - checking:A
2024-02-04 09:25:20,157 - DEBUG - 
Before list: []
2024-02-04 09:25:20,157 - DEBUG - 


Running: exclusive_list_check
2024-02-04 09:25:20,158 - DEBUG - before listing (List 1):[]
2024-02-04 09:25:20,158 - DEBUG - Should not be in (List 2):['C', 'G', 'F', 'E', 'B', 'D']
2024-02-04 09:25:20,159 - DEBUG - 
After list: ['B', 'D', 'G']
2024-02-04 09:25:20,159 - DEBUG - 


Running: exclusive_list_check
2024-02-04 09:25:20,159 - DEBUG - after listing (List 1):['B', 'D', 'G']
2024-02-04 09:25:20,159 - DEBUG - Should not be in (List 2):[]
2024-02-04 09:25:20,160 - DEBUG - Checking list: Sheet2
2024-02-04 09:25:20,160 - DEBUG - 


Running: account_order_checker
2024-02-04 09:25:20,160 - DEBUG - checking:A
2024-02-04 09:25:20,162 - DEBUG - 
Before list: [

In [15]:
ordered_account_list = correctly_ordered_list(check_account_order_dict)
ordered_account_list

2024-02-04 09:25:22,451 - DEBUG - 


Running: correctly_ordered_list
2024-02-04 09:25:22,451 - DEBUG - 
Assessing:C
2024-02-04 09:25:22,452 - DEBUG - before_list:['A', 'B']
2024-02-04 09:25:22,452 - DEBUG - max_before_pos:None
2024-02-04 09:25:22,453 - DEBUG - after_list:['G', 'E', 'F']
2024-02-04 09:25:22,453 - DEBUG - min_after_pos:None
2024-02-04 09:25:22,453 - DEBUG - Nones
2024-02-04 09:25:22,454 - DEBUG - ['C']
2024-02-04 09:25:22,454 - DEBUG - 
Assessing:G
2024-02-04 09:25:22,454 - DEBUG - before_list:['C', 'F', 'E', 'A', 'B', 'D']
2024-02-04 09:25:22,455 - DEBUG - max_before_pos:0
2024-02-04 09:25:22,455 - DEBUG - after_list:[]
2024-02-04 09:25:22,455 - DEBUG - min_after_pos:None
2024-02-04 09:25:22,456 - DEBUG - No after position indicator
2024-02-04 09:25:22,456 - DEBUG - max_before_pos:0
2024-02-04 09:25:22,456 - DEBUG - ['C', 'G']
2024-02-04 09:25:22,457 - DEBUG - 
Assessing:F
2024-02-04 09:25:22,457 - DEBUG - before_list:['C', 'A', 'B', 'E']
2024-02-04 09:25:22,458 - DEBUG

['A', 'B', 'D', 'C', 'E', 'F', 'G']

In [16]:
# Generate Excel with details on accounts
document_account_lists(ordered_account_list,account_lists_dict,)

2024-02-04 09:25:24,478 - DEBUG - 


Running: document_account_lists
2024-02-04 09:25:24,479 - DEBUG - excel_file_name:Ordered_Accounts_2024_02_04_at_09_25AM_.xlsx
2024-02-04 09:25:24,479 - DEBUG - excel_file_path:C:\Users\tyewf\github_projects\excel_accounting_data_analysis\results/Ordered_Accounts_2024_02_04_at_09_25AM_.xlsx
2024-02-04 09:25:24,481 - INFO - Excel saved as: Ordered_Accounts_2024_02_04_at_09_25AM_.xlsx


In [17]:
main(file_name = file_name)

2024-02-04 09:25:25,633 - DEBUG - 


Running: Main
2024-02-04 09:25:25,634 - DEBUG - 


Running: assert_file_extension
2024-02-04 09:25:25,637 - DEBUG - 


Running: read_sheets_as_df
2024-02-04 09:25:25,640 - DEBUG - 


Running: basic_checks_acc_gen
2024-02-04 09:25:25,640 - DEBUG - All checks have passes.
2024-02-04 09:25:25,640 - DEBUG - 


Running: get_list_of_unique_accounts
2024-02-04 09:25:25,640 - DEBUG - account_lists_dict:{'Sheet1': ['A', 'B', 'D', 'G'], 'Sheet2': ['A', 'B', 'C', 'E', 'F'], 'Sheet3': ['A', 'B', 'C', 'E', 'F', 'G']}
2024-02-04 09:25:25,642 - DEBUG - list of all unique accounts:['C', 'G', 'F', 'E', 'A', 'B', 'D']
2024-02-04 09:25:25,642 - DEBUG - 


Running: generate_check_account_order_dict
2024-02-04 09:25:25,642 - DEBUG - Current account listing:Sheet1
2024-02-04 09:25:25,643 - DEBUG - Current account listing:Sheet2
2024-02-04 09:25:25,643 - DEBUG - Current account listing:Sheet3
2024-02-04 09:25:25,643 - DEBUG - check_account_order_dict:{'C': {'before': ['A'

['A', 'B', 'D', 'C', 'E', 'F', 'G']

In [18]:
document_account_lists(ordered_account_list, account_lists_dict)

2024-02-04 09:25:30,282 - DEBUG - 


Running: document_account_lists
2024-02-04 09:25:30,283 - DEBUG - excel_file_name:Ordered_Accounts_2024_02_04_at_09_25AM_.xlsx
2024-02-04 09:25:30,284 - DEBUG - excel_file_path:C:\Users\tyewf\github_projects\excel_accounting_data_analysis\results/Ordered_Accounts_2024_02_04_at_09_25AM_.xlsx
2024-02-04 09:25:30,285 - INFO - Excel saved as: Ordered_Accounts_2024_02_04_at_09_25AM_.xlsx
